In [1]:
import code.profiles.generation as gen
from time import time
from code.utils import timed
from code.profiles.definitions import Profile, Voter, Candidate, VCR44
from code.profiles.vcrDetection import detectVCRProperty, detectCRProperty, detectVRProperty
from code.profiles.vcrDomain import isVCR
from code.profiles.consecutiveOnesDetection import consecutiveOnes2D, totalconsecutiveOnes2D

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext # sc = SparkContext.getOrCreate()
spark

In [3]:
def run(C=3, V=3):
    candidatesIds = ["A", "B", "C", "D"]
    votersIds = ["1", "2", "3", "4"]
    
    profilesRDD = sc.parallelize(gen.parallelProfileGeneration(cpu=16, candidatesNumber=C, voterNumber=V))
    
    print("ONE")
    
    vcrProfilesRDD = profilesRDD \
        .map(lambda p: (p,detectVCRProperty(A=p, C=candidatesIds[:C], V=votersIds[:V]))) \
        .filter(lambda pRes: pRes[1][0] == 2)

    print("VCR : ", vcrProfilesRDD.count())
    
    vcrNCOPProfilesRDD = vcrProfilesRDD \
        .filter(lambda pRes: not detectCRProperty(A=pRes[0], C=candidatesIds[:C], V=votersIds[:V])) \
        .filter(lambda pRes: not detectVRProperty(A=pRes[0], C=candidatesIds[:C], V=votersIds[:V]))

    print("NCOP VCR : ", vcrNCOPProfilesRDD.count())
    return vcrNCOPProfilesRDD

In [4]:
start = time()
vcrNCOPProfilesRDD = run(4,4)
end = time()
end - start

ONE
VCR :  57832
NCOP VCR :  96


53.51304006576538

In [5]:
vcrNCOPProfilesRDD.take(3)

[(array([[0, 0, 1, 1],
         [0, 1, 0, 1],
         [1, 0, 0, 1],
         [1, 1, 1, 1]]),
  (2,
   {'xA': 2.0,
    'xB': 4.0,
    'xC': 0.0,
    'xD': 0.0,
    'x1': 0.0,
    'x2': 4.0,
    'x3': 3.0,
    'x4': 0.0,
    'rA': 1.0,
    'rB': 0.0,
    'rC': 0.0,
    'rD': 2000000000.0,
    'r1': 0.0,
    'r2': 0.0,
    'r3': 0.0,
    'r4': 2000000000.0})),
 (array([[0, 0, 1, 1],
         [0, 1, 0, 1],
         [1, 1, 1, 1],
         [1, 0, 0, 1]]),
  (2,
   {'xA': 2.0,
    'xB': 4.0,
    'xC': 0.0,
    'xD': 0.0,
    'x1': 0.0,
    'x2': 4.0,
    'x3': 0.0,
    'x4': 3.0,
    'rA': 1.0,
    'rB': 0.0,
    'rC': 0.0,
    'rD': 2000000000.0,
    'r1': 0.0,
    'r2': 0.0,
    'r3': 2000000000.0,
    'r4': 0.0})),
 (array([[0, 0, 1, 1],
         [0, 1, 1, 0],
         [1, 0, 1, 0],
         [1, 1, 1, 1]]),
  (2,
   {'xA': 2.0,
    'xB': 4.0,
    'xC': 0.0,
    'xD': 0.0,
    'x1': 0.0,
    'x2': 4.0,
    'x3': 3.0,
    'x4': 0.0,
    'rA': 1.0,
    'rB': 0.0,
    'rC': 2000000000.0,
    

In [9]:
res = detectVCRProperty(all33Profiles[140], candidatesIds[:C], votersIds[:V])
result = res[1]
a = all33Profiles[140]
resV = [Voter(id=vId, x=result['x'+vId], r=result['r'+vId]) for vId in votersIds[:V]]
resC = [Candidate(id=cId, x=result['x'+cId], r=result['r'+cId]) for cId in candidatesIds[:C]]
profile = Profile(a, resV, resC)
isVCR(profile)

In [7]:
!cp README.md r.md